<a href="https://colab.research.google.com/github/Abdulkareem777/Abdulkareem/blob/main/Alexnet_on_cifar_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background-color:teal;color:white;padding:20px;  border-radius: 35px / 50px; text-align:center;">
  <h2 style="color:red";>Don't Forget To Up Vote If You Liked This Notebook </h2>
    </div>

## Importing Libraries

In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import cv2
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.datasets import cifar100

## Load Dataset

In [2]:
(X_train,y_train), (X_test, y_test) = cifar100.load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 19s 0us/step
X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [3]:
# Create a subplot with 3 rows and 3 columns
fig = make_subplots(rows=3, cols=3)

images = X_train[50:59]
# Add traces by row/col index
for i in range(9):
    fig.add_trace(
        go.Image(z=images[i]),
        row=i // 3 + 1, col=i % 3 + 1
    )

# Customize the layout of each subplot
for i in range(9):
    fig.update_xaxes(visible=False, row=i // 3 + 1, col=i % 3 + 1)
    fig.update_yaxes(visible=False, row=i // 3 + 1, col=i % 3 + 1)

# Adjust the size of images
fig.update_layout(
    images=[
        dict(
            source="",
            x=0,
            y=1,
            sizex=2 / 3,  # Adjust the size as needed
            sizey=2 / 3,  # Adjust the size as needed
            xref="paper",
            yref="paper",
            opacity=1.0
        )
    ]
,  width=900,
  height=900,)

# Show the figure
fig.show()

### Encoding labels

In [4]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

print(f"y_train shape after transformation: {y_train.shape}")
print(f"y_test shape after transformation: {y_test.shape}")

y_train shape after transformation: (50000, 100)
y_test shape after transformation: (10000, 100)


### Resizing Images

In [5]:
def resize_image(image):
    image = tf.image.resize(image, (224, 224))
    return image

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)

train_data = train_ds.map(lambda x, y: (resize_image(x), y))
test_data = test_ds.map(lambda x, y: (resize_image(x), y))

In [6]:
train_ds.take(1).as_numpy_iterator().next()[0].shape

(64, 32, 32, 3)


##  Data Augmantation

<div style="background-color:teal;color:white;padding:20px;  border-radius: 35px / 50px;">
<h2>In The paper:</h2>
<ul><h3><li>The apply horizontal reflection and PCA on image pixels</li>
<li>here we will do horizontal reflection and Rotation only</li></h3>
</ul>
</div>

In [7]:
def augmenter():
    """
    Creat a Sequential Model compose 2 layers
    Returns:
        tf.keras.Sequential
    """
    augmentation = tf.keras.Sequential()
    augmentation.add(tfl.RandomFlip("horizontal"))
    augmentation.add(tfl.RandomRotation(0.2))  # They did not include this in the paper

    return augmentation

In [8]:
data_augmentation = augmenter()

## AlexNet architecture

In [9]:
def Alex(input_shape, data_augmentation):

    input_img = tf.keras.Input(shape=input_shape)
    layer = augmenter()(input_img)

    layer = tfl.Conv2D(filters=96, kernel_size=11, strides=(4, 4))(layer)
    layer = tfl.ReLU()(layer)
    layer = tfl.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(layer)
    layer = tfl.BatchNormalization()(layer, training=True)

    layer = tfl.Conv2D(filters=256, kernel_size=5, strides=(1, 1))(layer)
    layer = tfl.ReLU()(layer)
    layer = tfl.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(layer)
    layer = tfl.BatchNormalization()(layer, training=True)

    layer = tfl.Conv2D(filters=384, kernel_size=3, strides=(1, 1))(layer)
    layer = tfl.ReLU()(layer)

    layer = tfl.Conv2D(filters=384, kernel_size=3, strides=(1, 1))(layer)
    layer = tfl.ReLU()(layer)

    layer = tfl.Conv2D(filters=256, kernel_size=3, strides=(1, 1))(layer)
    layer = tfl.ReLU()(layer)
    layer = tfl.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(layer)
    layer = tfl.BatchNormalization()(layer, training=True)

    layer = tf.keras.layers.Flatten()(layer)

    layer =tfl.Dense(2048, activation="relu")(layer)
    tfl.Dropout(0.5)
    layer = tfl.BatchNormalization()(layer, training=True)

    layer =tfl.Dense(2048, activation="relu")(layer)
    tfl.Dropout(0.5)
    layer = tfl.BatchNormalization()(layer, training=True)

    outputs = tf.keras.layers.Dense(100, activation='softmax')(layer)
    model = tf.keras.Model(inputs=input_img, outputs=outputs)

    return model

## Training and evaluating the model

In [10]:
model = Alex((224, 224, 3), data_augmentation)

model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [11]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 54, 54, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu (ReLU)                         │ (None, 54, 54, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 27, 27, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 27, 27, 96)          │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 23, 23, 256)         │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_1 (ReLU)                       │ (None, 23, 23, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 12, 12, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 10, 10, 384)         │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_2 (ReLU)                       │ (None, 10, 10, 384)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 8, 8, 384)           │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_3 (ReLU)                       │ (None, 8, 8, 384)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 6, 6, 256)           │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_4 (ReLU)                       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 3, 3, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 3, 3, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2048)                │       4,720,6

 Total params: 12,887,908 (49.16 MB)

 Trainable params: 12,878,500 (49.13 MB)

 Non-trainable params: 9,408 (36.75 KB)

<div style="background-color:teal;color:white;padding:20px;  border-radius: 35px / 50px;">
<h2>In The paper:</h2>
<ul>
    <h3><li>They trained The NN for <em >90 epochs </em> and this took <em>6 days</em> to finish the traing.</li>
        <br>
    <li>they expected that if they colud train it for <em >
more than 90 epochs </em> this will enhance the NN performance.</li>
        <br>
    <li>Here We will train it for just <em >
10 epochs</em>.</li></h3>
</ul>
</div>

In [12]:
epoch = 100
batch_size=128

history = model.fit(train_data, epochs=epoch,
                    validation_data=test_data,
                    batch_size=batch_size,
                    shuffle=True)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 42ms/step - accuracy: 0.0637 - loss: 4.6769 - val_accuracy: 0.0728 - val_loss: 5.1861
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 40ms/step - accuracy: 0.1337 - loss: 3.9980 - val_accuracy: 0.1739 - val_loss: 3.5571
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.1789 - loss: 3.6855 - val_accuracy: 0.1730 - val_loss: 3.7634
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.2005 - loss: 3.5321 - val_accuracy: 0.2093 - val_loss: 3.4941
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.2268 - loss: 3.1964 - val_accuracy: 0.2158 - val_loss: 611.5333
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.2617 - loss: 2.9644 - val_accuracy: 0.2544 - val_loss: 5.7972
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.2919 - loss: 2.7867 - val_accuracy: 0.2815 - val_loss: 4.8691
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.3168 - loss:

In [14]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create a subplot with two rows and one column
fig = make_subplots(rows=2, cols=1, subplot_titles=("Accuracy", "Loss"))

# Add training accuracy to the first subplot
fig.add_trace(go.Scatter(x=list(range(epoch)), y=history.history['accuracy'], mode='lines', name='Train Accuracy'), row=1, col=1)

# Add validation accuracy to the first subplot
fig.add_trace(go.Scatter(x=list(range(epoch)), y=history.history['val_accuracy'], mode='lines', name='Val Accuracy'), row=1, col=1)

# Add training loss to the second subplot
fig.add_trace(go.Scatter(x=list(range(epoch)), y=history.history['loss'], mode='lines', name='Train Loss'), row=2, col=1)


# Update the layout with the title
fig.update_layout(title='Model Accuracy and Loss')

# Show the figure
fig.show()
